# Music Generation using LSTMs

* Reference taken from [Karnika Kapoor - Kaggle](https://www.kaggle.com/karnikakapoor/music-generation-lstm/notebook)

In [4]:
import os
from music21 import *
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

## 1. Loading and parsing data


In [5]:
#Loading the list of chopin's midi files as stream
filepath = "./data/classical-music-midi/mozart/"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)